In [3]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2022
mmdd_str = today.strftime('%m%d')
mmdd_str

'0505'

In [4]:
#today = date(2022, 5, 3)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0505'

In [5]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  6


In [6]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  DOHOME
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070069950
Published:  Thu, 05 May 2022 21:57:25 +0700

----------------------------------

F45: 1
Title:  DOHOME
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070068600
Published:  Thu, 05 May 2022 20:01:40 +0700

----------------------------------

F45: 2
Title:  SMT
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070066090
Published:  Thu, 05 May 2022 19:05:12 +0700

----------------------------------

F45: 3
Title:  SPVI
Year:  2022
Quarter:  1
Link:  https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070055400
Published:  Thu, 05 May 2022 17:33:47 +0700

----------------------------------

F45: 4
Title:  TFM
Year:  2022
Quarter:  1
Link:  https://classic.set.or

In [7]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,DOHOME,2022,1,"Thu, 05 May 2022 21:57:25 +0700"
1,DOHOME,2022,1,"Thu, 05 May 2022 20:01:40 +0700"
2,SMT,2022,1,"Thu, 05 May 2022 19:05:12 +0700"
3,SPVI,2022,1,"Thu, 05 May 2022 17:33:47 +0700"
4,TFM,2022,1,"Thu, 05 May 2022 12:56:25 +0700"
5,LPN,2022,1,"Thu, 05 May 2022 12:37:54 +0700"


In [8]:
df.dtypes

name         object
year          int64
quarter      object
link         object
published    object
dtype: object

In [9]:
df.loc[(df.quarter == 'y') ,'quarter'] = '4'
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2022,1,6,6,6


In [14]:
df.loc[(df.quarter == 'e') ,'quarter'] = '4'
df.groupby(['year','quarter']).count()

name  link  published
year quarter                       
2022 1           5     5          5
     4           3     3          3

In [15]:
df.loc[df.quarter == '4','year'] = 2021
df.groupby(['year']).count()

,name,quarter,link,published
year,,,,
2021,3,3,3,3
2022,5,5,5,5


In [7]:
df.loc[df.quarter == 2,'year'] = 2021
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2021,4,2,2,2
2022,1,2,2,2


In [16]:
df.quarter = df.quarter.astype(int)
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2021,4,3,3,3
2022,1,5,5,5


In [ ]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == 'e','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['quarter']).count()

In [ ]:
df.loc[df.quarter == '1','quarter'] = 4
df.groupby(['quarter']).count()

In [ ]:
df.loc[df.quarter == ' ','quarter'] = 4
df.groupby(['quarter']).count()

In [ ]:
df.loc[df.quarter == 'S','quarter'] = 4
df.groupby(['quarter']).count()

In [ ]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == '1','year'] = 2022
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['year']).count()

In [10]:
df.shape

(6, 5)

In [11]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(5, 5)

In [12]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [13]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

405

In [14]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(408, 7)

### Tickers that won't be input

In [15]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
1,SMT,2022.0,1,"Thu, 05 May 2022 19:05:12 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070066090
2,SPVI,2022.0,1,"Thu, 05 May 2022 17:33:47 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070055400


In [16]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

2

In [17]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,DOHOME,2022.0,1,"Thu, 05 May 2022 21:57:25 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070069950
3,TFM,2022.0,1,"Thu, 05 May 2022 12:56:25 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070034170
4,LPN,2022.0,1,"Thu, 05 May 2022 12:37:54 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070033450


In [18]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

3

In [19]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [20]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(404, 14)

### There are no ticker records

In [21]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market
1,TFM,2022.0,1,"Thu, 05 May 2022 12:56:25 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070034170,NaN,NaN


In [22]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
].shape

(1, 7)

### There are ticker records

In [23]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
0,DOHOME,2022.0,1,"Thu, 05 May 2022 21:57:25 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070069950,701.0,SET100 / SETWB
2,LPN,2022.0,1,"Thu, 05 May 2022 12:37:54 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070033450,276.0,sSET / SETTHSI


In [24]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
0,DOHOME,2022.0,1,"Thu, 05 May 2022 21:57:25 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070069950,701.0,SET100 / SETWB
2,LPN,2022.0,1,"Thu, 05 May 2022 12:37:54 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070033450,276.0,sSET / SETTHSI


In [25]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
0,DOHOME,2022,1,"Thu, 05 May 2022 21:57:25 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070069950,701.0,SET100 / SETWB
2,LPN,2022,1,"Thu, 05 May 2022 12:37:54 +0700",https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070033450,276.0,sSET / SETTHSI


In [26]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [27]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(913, 14)

In [28]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(915, 19)

### Already input, display profit amt & eps to check with new F45

In [29]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge


In [30]:
df_merge3[df_merge3['_merge'] == 'both'].shape

(0, 19)

In [31]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3.shape

(2, 19)

In [32]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0    DOHOME 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070069950
1       LPN 2022 1 https://classic.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16517070033450
dtype: object